In [2]:
import os
import requests

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

api_key = os.environ['GROQ']
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

def get_completion(prompt, model="llama3-8b-8192"):
    messages = [{"role": "user", "content": prompt}]
    data = {
        "model": model,
        "messages": messages,
        "temperature": 0
    }

    response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=data)
    return response.json()["choices"][0]["message"]["content"]


In [3]:
print(get_completion("What is 1+1?"))

The answer to 1+1 is 2.


In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
"""

style = """American English in a calm and respectful tone
"""

prompt = f"""Translate the text
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

response = get_completion(prompt)
print(response)

Here is the translated text in American English with a calm and respectful tone:

"I'm really frustrated that my blender lid came loose and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now."

I've replaced the pirate slang and tone with a more neutral and respectful language, while still conveying the same level of frustration and need for assistance.


In [11]:
from langchain_groq import ChatGroq


chat = ChatGroq(model="llama3-8b-8192", temperature=0.0, api_key=api_key)
chat

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x788e7a78b5c0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x788e7bace0c0>, model_name='llama3-8b-8192', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [36]:
template_string = """Translate the text
that is delimited by triple backticks
into a style that is {style}.
text: ```{text}```
"""

In [37]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [38]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text\nthat is delimited by triple backticks\ninto a style that is {style}.\ntext: ```{text}```\n')

In [15]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [ ]:
customer_style = """American English
in a calm and respectful tone
"""

In [39]:
customer_email = """
Arrr, I be fuming that me blender lid 
flew off and splattered me kitchen walls 
with smoothie! And to make matters worse, 
the warranty don't cover the cost of 
cleaning up me kitchen. I need yer help 
right now, matey!
"""

In [18]:
customer_messages = prompt_template.format_messages(style=customer_style, text=customer_email)

In [19]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [20]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [21]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

/tmp/ipykernel_20703/1472147048.py:2: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  customer_response = chat(customer_messages)


In [22]:
print(customer_response.content)

Here is the translated text in American English, written in a calm and respectful tone:

"I'm really frustrated that my blender lid came loose and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now."

I've replaced the pirate slang and tone with a more neutral and polite language, while still conveying the same sense of frustration and need for assistance.


In [ ]:
service_reply = """Hey there customer, 
the warranty does not cover 
cleaning expenses for your kitchen \
because it's your fault that 
you misused your blender 
by forgetting to put the lid on before 
starting the blender. 
Tough luck! See ya!
"""

service_style_pirate = """
a polite tone 
that speaks in English Pirate
"""

In [24]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [25]:
service_response = chat(service_messages)
print(service_response.content)

Arrr, shiver me timbers! Here be the translated text, matey:

```Ahoy, dear customer! I be sorry to inform ye that yer warranty don't cover the cost o' cleanin' yer kitchen, mate. It seems ye got yerself into a bit o' trouble by forgettin' to put the lid on yer blender afore startin' it up. Aye, it be a mighty fine mess ye've gotten yerself into, but don't ye worry, I be here to help ye navigate the choppy waters o' responsibility! Ye see, it be yer duty to take care o' yer belongings, and neglectin' to follow the instructions be a mighty big no-no, savvy? So, I be afraid ye'll have to foot the bill yerself, matey. But don't ye worry, I be sure ye'll learn from yer mistakes and be more careful in the future. Fair winds and following seas to ye, me hearty!```


In [41]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

customer_review = """
This leaf blower is pretty amazing.  It has four settings:
candle blower, gentle breeze, windy city, and tornado. 
It arrived in two days, just in time for my wife's 
anniversary present. 
I think my wife liked it so much she was speechless. 
So far I've been the only one using it, and I've been 
using it every other morning to clear the leaves on our lawn. 
It's slightly more expensive than the other leaf blowers 
out there, but I think it's worth it for the extra features.
"""

review_template = """
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? 
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product 
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [42]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nFor the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? \nAnswer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product \nto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,\nand output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [43]:
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages)
print(response.content)

Here is the extracted information in JSON format:

```
{
    "gift": True,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```

Note that I assumed the item was purchased as a gift since it was mentioned as an "anniversary present". I also extracted the sentence about the price value as a comma-separated list.


In [29]:
type(response.content)

str

In [30]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

In [44]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days did it take for the product to arrive? If this  information is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any sentences about the value or  price, and output them as a  comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [45]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this  information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or  price, and output them as a  comma separated Python list.
}
```


In [46]:
review_template_2 = """
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? 
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [47]:
print(messages[0].content)
response = chat(messages)
print(response.content)
output_dict = output_parser.parse(response.content)
output_dict
type(output_dict)
output_dict.get('delivery_days')


For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? 
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,
and output them as a comma separated Python list.

text: 
This leaf blower is pretty amazing.  It has four settings:
candle blower, gentle breeze, windy city, and tornado. 
It arrived in two days, just in time for my wife's 
anniversary present. 
I think my wife liked it so much she was speechless. 
So far I've been the only one using it, and I've been 
using it every other morning to clear the leaves on our lawn. 
It's slightly more expensive than the other leaf blowers 
out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```j

OutputParserException: Got invalid JSON object. Error: Expecting value: line 2 column 10 (char 11)
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 